# Semantic Kernel

- The Semantic Kernel Agent Framework supports different types of agents, including `ChatCompletionAgent`, `OpenAIAssistantAgent`, and `AzureAIAgent`.
- Using the Semantic Kernel Agent Framework allows developers to quickly build agents on the Foundry Agent Service, with support for natural language processing and access to built-in tools in just a few lines of code.
- Semantic Kernel allows integration of Azure AI Agent capabilities, such as built-in tools and project deployment, without rewriting your code. 
- Semantic Kernel ensures consistent implementation across multiple types of agents.
- The Semantic Kernel Agent Framework plugins and functions feature allows your AI agent to interact with APIs, retrieve necessary data, and complete tasks.

## Semantic Kernel core components
The components can be used individually or combined:
- **AI service connectors**: connect the code to AI services from different providers under a common interface. Supported services include Chat Completion, Text Generation, and more.
- **Memory connectors**: expose vector stores from other providers under a common interface.
- **Functions and plugins**: containers for functions that are registered with the kernel. Once registered, functions can be invoked by the AI or through prompt templates.
- **Prompt templates**: combine instructions, user input, and function outputs into a reusable format. Prompt templates allow AI models to execute predefined steps dynamically.
- **Filters** - allow custom actions to be performed before and after a function or prompt is invoked. When registered, function filters act as outer layers and prompt filters as inner layers.

## Agent framework core concepts
- **Agent**: abstraction for AI agents, with specialized subclasses like AzureAIAgent.
- **Agent threads**:  manage conversation state and stores conversations.
- **Agent chat**: the foundation for multi-agent interactions, allows for structured conversations and collaboration.
- **Agent channel**: used for custom agent development, allows different types of agents to participate in AgentChat.
- **Agent messages**: a unified structure for agent communication, provides seamless communication and integration with existing AI workflows.
- **Templating**: like Semantic Kernel prompt templates, templates use dynamic prompt configurations to shape agent behavior.
- **Functions and plugins**: like Semantic Kernel plugins, agent plugin functions allow developers to extend agent capabilities by incorporating custom functions.

## AzureAIAgent key components
The Semantic Kernel AzureAIAgent object relies on the following components to function:
- `AzureAISAgentSettings`: an object that automatically includes the Azure AI Agent settings from the environment configuration. These settings will be used by the AzureAIAgents you create.
- `AzureAIAgent` client: an object that manages the connection to your Azure AI Foundry project.
- Agent service: the `AzureAIAgent` client also contains an agent operations service. This service helps streamline the process of creating, managing, and running the agents for your project.
- Agent definition: the AzureAI Agent model created via the AzureAI Project client, specifies the AI deployment model that should be used, and the name and instructions for the agent.
- `AzureAIAgentThread`: automatically maintains the conversation history between agents and users, and the state.

## With plugins

In Semantic Kernel, plugins work through function calling, allowing AI to request and use specific functions.

To enable automatic orchestration with function calling, plugins also need to provide details that describe how they behave. The function's input, output, and side effects should be described in a way that the AI can understand, otherwise, the AI will not correctly call the function.

# Setup

In [22]:
import os

from dotenv import load_dotenv

from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.functions import kernel_function
from typing import Annotated

load_dotenv()

AGENT_PROJECT_ENDPOINT = os.getenv("AGENT_PROJECT_ENDPOINT")
AGENT_MODEL_DEPLOYMENT = os.getenv("AGENT_MODEL_DEPLOYMENT")

In [23]:
# Plugin creation

class EmailPlugin:
    """A Plugin to simulate email functionality."""
    
    @kernel_function(description="Sends an email.")
    def send_email(self,
                   to: Annotated[str, "Who to send the email to"],
                   subject: Annotated[str, "The subject of the email."],
                   body: Annotated[str, "The text body of the email."]):
        print("\nTo:", to)
        print("Subject:", subject)
        print(body, "\n")

In [15]:
ai_agent_settings = AzureAIAgentSettings(
    endpoint=AGENT_PROJECT_ENDPOINT,
    model_deployment_name=AGENT_MODEL_DEPLOYMENT,
)

project_client = AzureAIAgent.create_client(
    endpoint=AGENT_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True
    )
)

In [ ]:
# Define an Azure AI agent that sends an expense claim email
expenses_agent_def = await project_client.agents.create_agent(
     model= ai_agent_settings.model_deployment_name,
     name="expenses_agent",
     instructions="""You are an AI assistant for expense claim submission.
                     When a user submits expenses data and requests an expense claim, use the plug-in function to send an email to expenses@contoso.com with the subject 'Expense Claim`and a body that contains itemized expenses with a total.
                     Then confirm to the user that you've done so."""
)

In [19]:
async for agent in project_client.agents.list_agents():
    print(agent.name)

expenses_agent


In [24]:
# Create a semantic kernel agent
expenses_agent = AzureAIAgent(
     client=project_client,
     definition=expenses_agent_def,
     plugins=[EmailPlugin()]
)

In [25]:
# Use the agent to process the expenses data
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: AzureAIAgentThread | None = None

prompt = """Here's my expense data:
date,description,amount
07-Mar-2025,taxi,24.00
07-Mar-2025,dinner,65.50
07-Mar-2025,hotel,125.90

Please notify by email.
"""

try:
     # Add the input prompt to a list of messages to be submitted
     prompt_messages = [prompt]
     # Invoke the agent for the specified thread with the messages
     response = await expenses_agent.get_response(prompt_messages, thread=thread)
     # Display the response
     print(f"\n# {response.name}:\n{response}")
except Exception as e:
     # Something went wrong
     print (e)
finally:
     # Cleanup: Delete the thread and agent
     await thread.delete() if thread else None
     await project_client.agents.delete_agent(expenses_agent.id)


To: expenses@contoso.com
Subject: Expense Claim
Itemized Expenses:
1. Date: 07-Mar-2025, Description: taxi, Amount: $24.00
2. Date: 07-Mar-2025, Description: dinner, Amount: $65.50
3. Date: 07-Mar-2025, Description: hotel, Amount: $125.90

Total Amount: $215.40 


# expenses_agent:
I've sent your expense claim email with the itemized expenses to expenses@contoso.com. Your total claim amount is $215.40.
